# Display image data on a web map <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Products used:** 
[s2a_nrt_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule)
* **Compatibility:** Notebook currently only compatible with `DEA Sandbox` environment

# Background

[Leaflet](https://leafletjs.com/) is the leading open-source JavaScript library for mobile-friendly interactive maps.
Functionality it provides is exposed to Python users by [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet), this library enables interactive maps in the Jupyter notebook/lab environment.

# Description

This notebook demonstrates how to plot an image and dataset footprints on a map.

1. Find some datasets to load
2. Load pixel data in EPSG:3857 projection, same as used by most web maps
3. Display dataset footptints on a map
4. Display image loaded from these datasets on the same map

### Load packages

In [ ]:
import numpy as np
from IPython.display import display
import ipyleaflet as L
from ipywidgets import widgets as w

import datacube
import odc.ui
from odc.ui import with_ui_cbk

### Configure datacube for efficient S3 access

This step is optional.
Hopefully in the future it will happen automatically.
Cell below configures internal libraries used by [Open Data Cube](https://www.opendatacube.org/) (GDAL and rasterio) in a way that allows more efficient data loading in the cloud environment (i.e. reading Cloud Optimized GeoTIFFs from S3)

In [ ]:
from datacube.utils.rio import set_default_rio_config
set_default_rio_config(aws={'region_name': 'auto'},
                       cloud_defaults=True)

### Connect to the datacube

In [ ]:
dc = datacube.Datacube()

## Find Datasets

In this example we are using Sentinel 2A NRT product.
We will be visualizing a portion of the swath taken by Sentinel 2A on 17-Oct-2019.

We want to display dataset footprints as well as imagery captured.
Rather than calling `dc.load(..)` directly with the time and spatial bounds we first use

```python
dss = dc.find_datasets(..)
``` 

to obtain a list of `datacube.Dataset` objects overlapping with our query first.

In [ ]:
# define product and red/green/blue bands in the given product
product = 's2a_nrt_granule'
RGB = ('nbar_red', 'nbar_green', 'nbar_blue')

# region and time of interest
query = dict(
    lat=(-34, -31),
    lon=(137, 138.5),
    time='2019-10-17',
)

dss = dc.find_datasets(product=product, **query)
print(f"Found {len(dss)} datasets")

## Load Red/Green/Blue bands

Since we already have a list of datasets (`dss`) we do not need to repeat the query, instead we supply datasets to `dc.load(.., datasets=dss, ..)` along with other parameters used for loading data.
Note that since we do not supply `lat/lon` bounds we will get all the imagery referenced by the datasets found earlier and the result will not be clipped to a `lat/lon` box in the query above.

We will load imagery at 200m per pixel resolution (1/20 of the native) in the Pseudo-Mercator (EPSG:3857) projection, same as used by most webmaps.

In [ ]:
rgb = dc.load(
    product=product,             # dc.load always needs product supplied, this needs to be fixed in `dc.load` code
    datasets=dss,                # Datasets we found for earlier
    measurements=RGB,            # Only load red,green,blue bands
    group_by='solar_day',        # Fuse all datasets captured on the same day into one raster plane
    output_crs='EPSG:3857',      # Default projection used by Leaflet and most webmaps
    resolution=(-200, 200),      # 200m pixels (1/20 of the native)
    resampling='bilinear',       # Use bilinear resampling when scaling down
    progress_cbk=with_ui_cbk())  # Display load progress
rgb

### Create Leaflet Map with dataset footprints

`odc.ui.show_datasets` will create a Leaflet map and add dataset footprints to it.
Note that we will not be displaying the map just yet.

In [ ]:
m = odc.ui.show_datasets(
    dss,
    style={
        'opacity': 0.3,     # Footprint outline opacity
        'fillOpacity': 0    # Do not fill
    },
    width='600px',           # Set Width of the map to 600 pixels, examples: "100%", "5em", "300px"
    height='600px',          # Height of the map
    scroll_wheel_zoom=True)  # Allow zoom with the mouse scroll wheel

### Create Leaflet Image Layer

Under the hood `mk_image_layer` will:

1. Convert 16-bit `rgb` xarray to an 8-bit RGBA image
2. Encode RGBA image as PNG data `odc.ui.to_rgba`
3. Render PNG data to "data uri"
4. Compute image bounds
5. Construct `ipyleaflet.ImageLayer` with uri from step 3 and bounds from step 4

JPEG compression can also be used (e.g `fmt="jpeg"`), useful for larger images to reduce notebook size in bytes (use `quality=40` to reduce size further), downside is no opacity support unlike PNG.

Satellite imagery is often 12-bit and higher, but web images are usually 8-bit, hence we need to reduce bit-depth of the input imagery such that there are only 256 levels per color channel.
This is where `clamp` parameter comes in. In this case we use `clamp=3000`.
Input values of `3000` and higher will map to value `255` (largest possible 8-bit unsigned value), `0` will map to `0` and every other value in between will scale linearly.

In [ ]:
img_layer = odc.ui.mk_image_overlay(
    rgb,
    clamp=3000,  # 3000 -- brightest pixel level
    fmt='png')   # "jpeg" is another option

# add image layer to a map we created earlier
m.add_layer(img_layer)

### Add Opacity Control

- Add Vertical Slider to the map
- Dragging slider down lowers opacity of the image layer
- Use of `jslink` from `ipywidgets` ensures that this interactive behaviour will work even on a pre-rendered notebook (i.e. on nbviewer)

In [ ]:
slider = w.FloatSlider(min=0, max=1, value=1,        # Opacity is valid in [0,1] range
                       orientation='vertical',       # Vertical slider is what we want
                       readout=False,                # no need to show exact value
                       layout=w.Layout(width='2em')) # Fine tune display layout: make it thinner

# Connect slider value to opacity property of the Image Layer
w.jslink((slider, 'value'),         
         (img_layer, 'opacity') )
m.add_control(L.WidgetControl(widget=slider))

### Finally display the map

In [ ]:
display(m)

### Sharing Notebooks Online

Unlike notebooks with `matplotlib` figures, saving notebook with Maps after running it is not enough to have results displayed when sharing rendered notebooks online.
You also need to save "Widget State" as a separate step, and you need to make sure that "Widget State" is saved only once within the notebook.
There are some unexpected behaviours one have to work around.
At the time of writing this only works in `jupyter notebook` not `jupyter lab`.

A robust algorithm for sharing notebooks with widget state is as following:

- Make sure there is no state/output in the notebook
    1. Open notebook in `jupyter notebook` not `lab`
    2. Run Menu: `Kernel > Restart and Clear Output`
    3. Run Menu: `Widgets > Clear Notebook Widget State`
    4. Save notebook
    5. Run Menu: `File > Close and halt`
    6. Verify from filesystem that notebook size is small
- Starting from clean notebook do
    1. Run Menu: `Cell > Run All`
    2. Run Menu: `Widgets > Save Notebook Widget State`
    3. Save notebook
    4. Run Menu: `File > Close and halt`
    
Following procedure above will ensure that widget state is stored in the notebook, and also that only relevant widget state is stored.
If you re-run any cells and then save widget state you might end up with stale widget state present in the notebook document as well, that state won't be displayed but will significantly increase your notebook on-disk size, preventing tools like nbviewer from rendering it.
The important thing to remember is that `clear widget state` only removes it from the notebook file, not from your browser, so next time you run `save widget state` stale state will re-appear on disk.
To make sure that stale `widget state` is properly gone you have to reload the notebook in your browser after clearing it.

------------------------------------------------